In [2]:
from time import sleep
import json
import os

def delay(delay_msec = 100):
    dieTime = QtCore.QTime.currentTime().addMSecs(delay_msec)
    while (QtCore.QTime.currentTime() < dieTime):
        QtCore.QCoreApplication.processEvents(QtCore.QEventLoop.AllEvents, 100)
def flip_powermirror():
    pulser.set_channel_off(['ch5'])
    pulser.set_channel_on(['ch5'])

# flip_powermirror()

# ello_flip.move_pos_1()

# pulser.set_channel_on(['ch6'])

# powermeter.get_power() * 1e6

bin_width = int(1e12/50)
n_vals = 300
def calibrate(steps = [240, 280, 300, 320, 360]):
    calibration = {}
    for step in steps:
        motor_pi3.moveToAbsolutePosition(motor=0, pos=step)
        delay(3000)
#         flip_powermirror()
        delay(2000)
        for i in range(2):
            try:
                pwr = powermeter.get_power() * 1e6
            except:
                pass
            delay(200)
#         flip_powermirror()
        calibration.update({step:pwr})
    return calibration


import os
import time
import datetime
from qtpy import QtCore
import numpy as np
from core.connector import Connector
from core.configoption import ConfigOption
from logic.generic_logic import GenericLogic


class Automatedmeasurement(QtCore.QThread):
    """ How to use this thing:
        1. create pois with poimanager (Qudi)
        2. create instance of class
        3. specify steps with "init_steps()"
        4. start with "start()"
            4.1 stop with "stop()"
        If pois have changed since class was initiated, update pois with "init_pois()"
            
        How to add new steps:
        1. write function that starts what you want to do (e.g. take a spectrum)
        2. add it to the func_dict in __init__()
        3. look for the signal that gets emitted once the step is done or create it yourself
        4. connect it to _next_step() in __init__()
    """
    # declare connectors
#     scannerlogic = Connector(interface='ConfocalLogic')
#     spectrumlogic = Connector(interface='SpectrumLogic')
#     optimizerlogic = Connector(interface = 'OptimizerLogic')
#     nicard = Connector(interface='NationalInstrumentsXSeries', optional=True)
#     ello_flipp = Connector(interface='ThorlabsElloFlipper', optional=True)
#     steps = ConfigOption('steps', missing='warn')
    # internal signals

    sigNextPoi = QtCore.Signal()
    sigNextStep = QtCore.Signal()
    sigStepDone = QtCore.Signal()
    abort = False
    steps = ['move', 'optimize', 'spectrum', 'time_trace']
    steps_bg = ['move', 'spectrum', 'time_trace']
    def __init__(self):
        super().__init__()
        self.func_dict = {
            'move' : self.move_to_poi,
            'optimize' : self.optimize_on_poi,
            'spectrum' : self.take_spectrum,
            'time_trace' : self.measure_time_trace
        }
        self.on_activate()
    def on_activate(self):
        """ Initialisation performed during activation of the module.
        """
 
        self._spectrumlogic = spectrumlogic
        self._scannerlogic = scannerlogic
        self._optimizerlogic = optimizerlogic
        # self._poimanagerlogic = self.poimanagerlogic()
        # self._save_logic = self.savelogic()
#         self._ello_flipper = ello_flip
#         if self.ello_devices():
#             self._ello_rotor = self.ello_devices().ello_rotor
        # self._cwave = self.cwavelaser()
        # self.sig_cwave_shutter.connect(self._cwave.set_shutters_states)

        # self._nicard = self.nicard()
        self.save_folder = save_folder0
        
        ## signals
        # connect internal signals
        self.sigNextPoi.connect(self._next_poi, QtCore.Qt.QueuedConnection)
        self.sigNextStep.connect(self._next_step,QtCore.Qt.QueuedConnection )
        # connect signals that mark the completion of one step
        self.sigStepDone.connect(self._next_step,QtCore.Qt.QueuedConnection )
        spectrumlogic.sig_specdata_taken.connect(self.save_spectra, QtCore.Qt.QueuedConnection)
        #spectrumlogic.sig_specdata_updated.connect(self._next_step)
        
        self._optimizerlogic._sigFinishedAllOptimizationSteps.connect(self._next_step, QtCore.Qt.QueuedConnection)

        ## initialisation of variables
        self.angles_for_pol_dep_spec = np.linspace(0,360,3) # TODO: change num to 100
    
    def on_deactivate(self):
        pass

    def start(self):
        """Starts the measurements"""
        self.abort = False
        self.init_pois()
        self.sigNextPoi.emit()
        return
    
    
    def stop(self):
        """Stops the program"""
        self.abort = True
        return
    
    
    def init_pois(self,poi_names=None):
        if poi_names==None:
            self.poi_names = list((np.arange(len(self._scannerlogic.pois)) + 1).astype(str))#self._poimanagerlogic.poi_names
        else:
            self.poi_names = poi_names
        # copy the names into an array that we can modify
        self._poi_names = list(self.poi_names).copy() # shallow copy
        pois = self._scannerlogic.pois
        texts = (np.arange(len(self._scannerlogic.pois)) + 1).astype(str)
        self.poi_positions = {t: pos for t, pos in zip(texts, pois)} #self._poimanagerlogic.poi_positions
        return
    
    def init_steps(self,steps):
        """Stores the steps as class object."""
        keys = self.func_dict.keys()
        if not set(steps).issubset(set(keys)):
            raise Exception('The following steps are not listed in the func_dict: %s'% (list(set(steps) - set(keys))))
        return
    
    @QtCore.Slot()
    def _next_poi(self):
        """Iterates through the pois.
        
        Sets the next poi to be the active one and starts the measurements on this one."""
        print('poi')
        # Stop program if finished or user wants to stop
        if self.abort or (len(self._poi_names)==0):
            print('Stopping(poi).')
            print(self.abort,len(self._poi_names))
            return
        # Choose the first poi in the list, set it as the current one and delete it.
        self._current_poi_name = self._poi_names.pop(0)
        print('Current poi %s'%self._current_poi_name)
        # updates the position of the current poi
        self._current_poi_position = self.poi_positions[self._current_poi_name]
        # copy the steps into an array that we can modify
        if self._current_poi_name != '1':
            self._steps = self.steps.copy() #shallow copy
        else:
            self._steps = self.steps_bg.copy()
        print(self._steps)
        # start the steps
        self.sigNextStep.emit()
        return
    @QtCore.Slot()
    def _next_step(self):
        """Iterates through the steps."""
        print('step')
        if self.abort:
            print('Stopping (step).')
            return
        if len(self._steps)==0:
            # all steps for the current poi are done, go to next poi
            self.sigNextPoi.emit()
            return
        # choose next step in list as current one and remove it
        self._current_step = self._steps.pop(0)
        print('Current step: %s'%self._current_step)
        #run the step
        self.func_dict[self._current_step]()
        
        #motor_pi3.moveToAbsolutePosition(motor=0, pos=310)
        delay(1000)
        return
    
    def move_to_poi(self,poi_name=None,rs=1000):
        """Moves to the current poi"""
        print('move')
        if poi_name==None:
            poi_name = self._current_poi_name
            poi_position = self._current_poi_position
        else:
            poi_position = self.poi_positions[poi_name]
        if rs == None:
            # get return slowness from confocal logic 
            rs = self._scannerlogic.return_slowness
        print(poi_name)
        # script will move to next line once position is reached

        self._scannerlogic.go_to_position('scanner', x=poi_position[0],y=poi_position[1],z=poi_position[2], rs=rs)
        self.current_poi_position = poi_position
        # no signal is emitted once position is reached. 
        # We need to send one by ourself to keep consistency with the rest of the script.
        
        self.sigStepDone.emit()
        return
    
    
    def optimize_on_poi(self):
        """Tells optimizerlogic to start the refocus.
        
        On completion, a signal is emitted from optimizerlogic
        """
        delay(200)
        confocal.refocus_clicked() 
        return
    
    
    def take_spectrum(self):
        """Tells spectrumlogic to start taking a spectrum.
        
        """
        delay(2000)
        spectrometer._mw.rec_single_spectrum_Action.trigger()
        return
    @QtCore.Slot()
    def save_spectra(self):
        self._spectrumlogic.save_spectrum_data(filepath=self.save_folder, name_tag = self._current_poi_name + '_575LP_spectrum_inttime_' + str(5))
        self.sigStepDone.emit()
        
        
    def measure_time_trace(self):
#         tt_counter.set_up_correlation(bins_width = 200, n_values = 1000)
        for step, power in calibration.items():
            motor_pi3.moveToAbsolutePosition(motor=0, pos=int(step))
            delay(2000)
            count_params = {
                "channels":[0, 1],
                "bins_width": bin_width,
                "n_values":n_vals
                }
            corr_params = {
                "channels":[0, 1],
                "bins_width": 500,
                "n_values":1000
                }
            
            timetagger.set_counter(count_params)
            delay(2000)
            counts = timetagger.countrate  * tt_counter._count_frequency
            savelogic.save_array_as_text(counts, filename=self._current_poi_name + '_timetrace_n' + str(n_vals) + '_'+ str(step) + '_' + '.txt',filepath=self.save_folder)
        #get correlation
        motor_pi3.moveToAbsolutePosition(motor=0, pos=int(list(calibration.keys())[-1]))
        delay(2000)
        timetagger.set_correlation(corr_params)
        delay(10000)
        dt, corrs = timetagger.corr_data
        corr_data = np.vstack((dt, corrs))
        savelogic.save_array_as_text(corr_data, filename=self._current_poi_name + '_correlation_' + '_'+ str(list(calibration.keys())[-2]) + '_' + '.txt',filepath=self.save_folder)
        self.sigStepDone.emit()
        
    def take_pol_dependent_spectrum(self):
        """ Starts a polarization dependent spectrum measurement.

        Tells the ello to rotate (the lambda/2 plate).
        And then tells spectrumlogic to start taking a spectrum.
        """
        print('pol dep spec')

        angles = self.angles_for_pol_dep_spec

        for angle in angles:
            poi_name = self._current_poi_name
            name_tag = poi_name + '_' + str(angle)
            if self._ello_rotor is not None:
                self._ello_rotor.move_abs(to_angle=angle)
            self._spectrumlogic.get_single_spectrum()
            self._spectrumlogic.save_spectrum_data(name_tag=name_tag)
        self.sigStepDone.emit()
        return

In [7]:
# scannerlogic.pois = np.array([])

In [32]:
save_folder0 = r"I:\data\Vlad\197\charge_study_1500\4p\Pb_p_etched_40\measurements"

In [10]:
# with open(os.path.join(save_folder0, "calibration.txt"), 'r') as file:
#      calibration = json.load(file) # use `json.loads` to do the reverse

In [4]:
flip_powermirror()

In [9]:
motor_pi3.moveToAbsolutePosition(motor=0, pos=300)

In [6]:
1e6 * powermeter.get_power()

0.105723913

In [3]:
calibration = calibrate(np.linspace(0, 360, 100).astype(int))
# motor_pi3.moveToAbsolutePosition(motor=0, pos=310)
calibration

{0: 0.0296166434, 3: 0.0291623081, 7: 0.029500881700000002, 10: 0.0296788834, 14: 0.029414994899999998, 18: 0.0296228677, 21: 0.029495904400000002, 25: 0.029178487200000002, 29: 0.029588015600000003, 32: 0.0296253582, 36: 0.0295544069, 40: 0.030236538600000005, 43: 0.0298979614, 47: 0.0294373983, 50: 0.0293826297, 54: 0.030245249900000003, 58: 0.0293502644, 61: 0.0295369809, 65: 0.0305078949, 69: 0.0295780573, 72: 0.029493413899999998, 76: 0.0303000185, 80: 0.0294299305, 83: 0.0294237061, 87: 0.030566401000000003, 90: 0.0296378069, 94: 0.0295780573, 98: 0.030459350599999997, 101: 0.0294013027, 105: 0.029372671399999998, 109: 0.0303224255, 112: 0.0295232887, 116: 0.029540714800000003, 120: 0.0304518792, 123: 0.0293378193, 127: 0.0293415532, 130: 0.0294784748, 134: 0.0292506854, 138: 0.029498391300000003, 141: 0.029489679999999997, 145: 0.029428687000000002, 149: 0.0293539983, 152: 0.0263877311, 156: 0.0263777729, 160: 0.0263192668, 163: 0.0265271431, 167: 0.0265445692, 170: 0.0265296335

In [26]:
calibration

{0: 240.241992, 3: 260.690023, 7: 281.55898299999996, 10: 307.04643000000004, 14: 382.026279, 18: 457.063114, 21: 503.989693, 25: 621.961779, 29: 788.5139780000001, 32: 940.184225, 36: 1191.75157, 40: 2448.2200399999997, 43: 2696.74067, 47: 2788.66873, 50: 2746.98739, 54: 2723.0060700000004, 58: 2588.82553, 61: 2567.12828, 65: 2512.88502, 69: 2534.01138, 72: 2554.56683, 76: 2539.14995, 80: 2544.28876, 83: 2649.34939, 87: 2595.67704, 90: 2504.89102, 94: 2542.57582, 98: 2552.28276, 101: 2460.35471, 105: 2467.20668, 109: 2496.32658, 112: 2472.91615, 116: 2437.51542, 120: 2516.3108899999997, 123: 2301.7383600000003, 127: 2196.13942, 130: 1875.69344, 134: 0.24624120000000002, 138: 0.265670565, 141: 0.286528461, 145: 0.309557947, 149: 0.33761614300000004, 152: 0.36710301300000003, 156: 0.390132499, 160: 0.41419056600000004, 163: 0.45076339499999996, 167: 0.490307798, 170: 0.520537526, 174: 0.58053979, 178: 0.642827956, 181: 0.655799852, 185: 0.685001055, 189: 0.753575193, 192: 0.857407656, 1

In [91]:
with open(os.path.join(save_folder0, "calibration.txt"), 'w') as file:
     file.write(json.dumps(calibration)) # use `json.loads` to do the reverse

In [10]:
motor_pi3.zeroMotor()

In [90]:
motor_pi3.moveToAbsolutePosition(motor=0, pos=110)

In [93]:
optimizerlogic._max_offset_z = 1

In [94]:
optimizerlogic._max_offset_xy = 1

In [33]:
auto_measurement = Automatedmeasurement()

In [34]:
auto_measurement.steps = ['move', 'optimize', 'spectrum']
auto_measurement.steps_bg = ['move', 'spectrum']

In [30]:
auto_measurement.stop()

In [20]:
scannerlogic.pois = np.array([])

In [31]:
auto_measurement = None

In [35]:
auto_measurement.start()

In [ ]:
file = os.path.join(save_folder, '1_timetrace_n3000_360_.txt') 

In [15]:
fold = r'I:\data\Vlad\197\1500\cross pplus_etched before annealing'

In [36]:
spectras = [spec for spec in os.listdir(save_folder0) if 'spectrum' in spec and '575LP' in spec and '.dat' in spec]

In [15]:
# countrate = np.genfromtxt(os.path.join(fold, '2_timetrace_n3000_360_.txt'))

In [37]:
import os

In [38]:
from matplotlib import pyplot as plt

In [74]:
spectra_datas = np.array([])
lam = np.genfromtxt(os.path.join(save_folder0, spectras[0]))[:,0]
signal_bg = np.genfromtxt(os.path.join(save_folder0, spectras[0]))[:,2]
for spec in spectras:
    lam = np.genfromtxt(os.path.join(save_folder0, spec))[:,0]
    signal = np.genfromtxt(os.path.join(save_folder0, spec))[:,2]
    
    spectra_datas = np.concatenate((spectra_datas, signal - signal_bg))
spectra_datas_nd = spectra_datas.reshape(len(spectras), signal_bg.shape[0])

## Spectra map

In [39]:
SPEC_TYPE=np.dtype([('wl',np.float),('crr',np.float)])

In [40]:
from matplotlib import ticker

In [41]:
scan_num = 0
spectrum_range = [540, 720]
scan_map = np.array([])
for idx, scan in enumerate(spectras):
    scan_num += 1
    spec = np.genfromtxt(os.path.join(save_folder0, scan), dtype = SPEC_TYPE)
    spec = spec[(spec['wl'] > spectrum_range[0]) & (spec['wl'] < spectrum_range[1])]
    scan_map = np.concatenate((scan_map, spec['crr']))
scan_map_r = scan_map.reshape((scan_num, spec['crr'].shape[0]))

fig, ax = plt.subplots(1, figsize=(10, 5))
ax.imshow(scan_map_r, 
          aspect='auto', 
          extent=[spec['wl'][0], spec['wl'][-1],scan_map_r.shape[0], 0])
ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5))
ax.set_xlabel(r'$\lambda$, nm')
ax.set_ylabel(r'defect #')
ax.grid(None)
plt.grid(None)
plt.title('SnV Reference')
plt.savefig(f"{save_folder0}\\spectra_map.png", dpi= 200)

In [42]:
fig, ax = plt.subplots(1, figsize=(10, 5))
ax.imshow(scan_map_r, 
          aspect='auto', 
          extent=[spec['wl'][0], spec['wl'][-1],scan_map_r.shape[0], 0])#, vmin=1000)
ax.xaxis.set_major_locator(ticker.MultipleLocator(10))
ax.xaxis.set_minor_locator(ticker.MultipleLocator(5))
ax.set_xlabel(r'$\lambda$, nm')
ax.set_ylabel(r'defect #')
plt.grid(None)
ax.grid(None)
plt.title('SnV Reference')
plt.show()

In [43]:
plt.plot(scan_map_r.sum(0))
plt.show()

In [45]:
plt.imshow(scan_map_r, 
          aspect='auto', 
          extent=[spec['wl'][0], spec['wl'][-1],scan_map_r.shape[0], 0])
plt.show()

In [46]:
from matplotlib import pyplot as plt